Start by importing all the modules to be used

In [ ]:
import tango
import time
import json
import os

Set up device proxies for our tango devices and put them into admin mode (allowing them to be used)

In [ ]:
# create device proxies and put devices in Online, not simulated mode
sr=tango.DeviceProxy('low-mccs-spshw/subrack/0001')
sr.adminMode=0

tpm1=tango.DeviceProxy("low-mccs-spshw/tile/0003")
tpm1.adminMode = 0

daq = tango.DeviceProxy("low-mccs/daqreceiver/001")
daq.adminMode = 0

# turn on subrack and wait for ON status
sr.On()
while sr.state() != tango._tango.DevState.ON:
    time.sleep(1)

# turn on DAQ and wait for ON status
daq.On()
while daq.state() != tango._tango.DevState.ON:
    time.sleep(1)

# turn on TPM and wait for initialization. 
tpm1.On()
while tpm1.tileprogrammingstate != "Initialised" and \
	tpm1.tileprogrammingstate != "Synchronised":
	time.sleep(1)

Set up the data stream on the tile and daq

In [ ]:
# Start TPM acquisition and wait for ADC stream to start
tpm1.startacquisition('{}')
while tpm1.tileprogrammingstate != "Synchronised":
	time.sleep(1)

#configure networking between TPM and DAQ.
dst_tpm_port = 4660
dst_tpm_ip = "10.0.10.210"
lmc_download_dict = {"mode": "10g", "destination_ip": dst_tpm_ip, "destination_port": dst_tpm_port}
tpm1.SetLmcDownload(json.dumps(lmc_download_dict))

Start the daq

In [ ]:
daq.Start('{"modes_to_start": "DaqModes.RAW_DATA"}')

data_to_send = {"data_type": "raw"}
tpm1.SendDataSamples(json.dumps(data_to_send))

Show that we can monitor health attributes from jupyter notebook

In [ ]:
tgm -w 1200 low-mccs-spshw/tile/0003/voltage